In [15]:
import pandas as pd
import numpy as np
import yfinance as yf
import module1 
import matplotlib.pyplot as plt
import plotly.express as px

In [29]:
ticker_list = ["AAPL", "MSFT", "AMZN", "NVDA", "GOOGL", "TSLA", "META", "JPM"]
start = "2022-01-01"
end = "2022-05-01"

df = pd.DataFrame(yf.download(ticker_list, start, end))

[*********************100%%**********************]  8 of 8 completed


In [38]:
data2 = df['Adj Close'].pct_change()

fig = px.line(data2, data2.index, [data2[ticker] for ticker in ticker_list], title='Stock returns')
fig.update_layout(title="Stock returns", legend_title="Stocks")
fig.update_yaxes(title_text="Returns")
fig.show()

/usr/local/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

